# Ticket report


In [1]:
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

from datetime import date
from ipywidgets import interact, interact_manual
from src.db.utils import SnowflakeWrapper
from src.compute.utils import Interval
from src.compute.tickets import get_tickets
conn = SnowflakeWrapper.create_snowflake_connection()
sw = SnowflakeWrapper(conn)

label_map = {
    "metrics": {
        "Average hours": "AVG_HOUR",
        "Average days": "AVG_DAYS",
        "Degree of cycling": "DegreeOfCycling",
    },
    "dims": {
        "Ticket Status": "MERGED_STATUS",
        "Ticket Type": "IssueType",
        "Ticket Priority": "IssuePriority",
        "No color": None
    }
}

In [2]:
def plot_distribution_counts(x_col, color_by, date_from, date_to):
    current_interval = Interval(date_from, date_to)
    plot_df = get_tickets(sw, current_interval)
    return px.histogram(
        plot_df,
        x=label_map["dims"][x_col],
        color=label_map["dims"][color_by],
        barmode="group",
        title=f"Resolved tickets [{len(plot_df.index)}] on the interval {current_interval.pretty_str()}",
        hover_data=plot_df.columns,
        labels = {
            label_map["dims"][x_col]: x_col,
            label_map["dims"][color_by]: color_by,
            "count": "Number of tickets"
        }
    )

_ = interact(
        plot_distribution_counts,
        x_col=widgets.Dropdown(options=label_map["dims"].keys()),
        color_by=widgets.Dropdown(options=label_map["dims"].keys()),
        date_from=widgets.DatePicker(value=pd.to_datetime('2019-10-01')),
        date_to=widgets.DatePicker(value=pd.to_datetime('2020-01-01'))
    )

interactive(children=(Dropdown(description='x_col', options=('Ticket Status', 'Ticket Type', 'Ticket Priority'…

In [6]:
def plot_metric_counts(x_col, color_by, date_from, date_to):
    current_interval = Interval(date_from, date_to)
    plot_df = get_tickets(sw, current_interval)
    bins = int(plot_df[label_map["metrics"][x_col]].max() - plot_df[label_map["metrics"][x_col]].min())
    max_bins = 30
    print(f"bins = {bins}")
    if label_map["metrics"][x_col] == "AVG_HOUR":
        bins = int(bins/8)
    if bins > max_bins:
        plot_df[label_map["metrics"][x_col]] = plot_df[label_map["metrics"][x_col]].apply(lambda x: min(max_bins, x))
        bins = max_bins
    print(f"bins = {bins}")
    return px.histogram(
        plot_df,
        x=label_map["metrics"][x_col],
        color=label_map["dims"][color_by],
        nbins=bins,
        barmode="group",
        title=f"Resolved tickets [{len(plot_df.index)}] on the interval {current_interval.pretty_str()}",
        hover_data=plot_df.columns,
        template="ggplot2",
        labels = {
            label_map["metrics"][x_col]: x_col,
            label_map["dims"][color_by]: color_by,
            "count": "Number of tickets"
        }
    )

_ = interact(
        plot_metric_counts,
        x_col=widgets.Dropdown(options=label_map["metrics"].keys()),
        color_by=widgets.Dropdown(options=label_map["dims"].keys()),
        date_from = widgets.DatePicker(value=pd.to_datetime('2019-10-01')),
        date_to = widgets.DatePicker(value=pd.to_datetime('2020-01-01'))
    )


interactive(children=(Dropdown(description='x_col', options=('Average hours', 'Average days', 'Degree of cycli…